In [1]:
import sys, os
sys.path.append(os.path.abspath(
    os.getcwd()+"/src")
)

In [2]:
from pymatching import Matching
from qecsim.models.rotatedplanar import RotatedPlanarCode, RotatedPlanarPauli
from qecsim.models.generic import BiasedDepolarizingErrorModel
import jax.numpy as jnp
from jax import random
from src.data_gen import noise_permutations_from_deformation, sample_errors
from src.recursive_mwpm import recursive_mwpm

key = random.key(42)

In [3]:
L = 13
bias = 500
error_probability = 0.1

code = RotatedPlanarCode(L, L)
noise_model = BiasedDepolarizingErrorModel(bias=bias, axis='Y')
probabilities = jnp.array(noise_model.probability_distribution(error_probability))

deformation = jnp.zeros((L, L), dtype=jnp.int32)
deformation = random.randint(key, (L, L), 0, 6, dtype=jnp.int32)
noise_permutations = noise_permutations_from_deformation(deformation)
deformed_probabilities = probabilities[noise_permutations].reshape((code.n_k_d[0], 4))
print(deformation)

wx = -jnp.log(deformed_probabilities[:,1])
wy = -jnp.log(deformed_probabilities[:,2])
wz = -jnp.log(deformed_probabilities[:,3])

[[0 0 3 1 5 3 5 3 4 0 0 4 1]
 [3 2 4 1 4 1 5 4 1 4 5 0 5]
 [2 1 0 5 3 0 1 2 1 2 4 0 3]
 [1 1 0 3 1 5 2 2 2 5 5 4 5]
 [0 3 4 3 1 4 5 1 1 4 4 4 1]
 [3 4 2 4 3 4 3 5 1 2 4 4 4]
 [5 5 1 4 5 4 3 2 0 2 4 2 3]
 [1 2 4 1 1 3 3 2 3 1 4 5 4]
 [1 2 4 4 3 3 5 1 3 2 1 5 3]
 [2 4 5 3 0 2 3 2 2 2 2 3 4]
 [1 4 0 1 1 4 5 0 5 3 5 2 1]
 [4 2 0 1 3 5 3 1 0 3 0 4 1]
 [2 5 3 4 4 0 0 5 4 1 3 1 2]]


In [19]:
noise_permutations.shape

(13, 13, 4)

In [15]:
subkey, key = random.split(key)

error = sample_errors(subkey, code, noise_model, error_probability, noise_permutations)

syndrome = code.stabilizers @ error
logicals = code.logicals @ error

error_z, error_x = error.reshape((2,-1))
error_weight = sum([
    wx[jnp.where(jnp.logical_and(error_x==1, error_z==0))].sum(),
    wy[jnp.where(jnp.logical_and(error_x==1, error_z==1))].sum(),
    wz[jnp.where(jnp.logical_and(error_x==0, error_z==1))].sum(),
])
print(f"Error weight: {error_weight:.2f}"+50*' ')
print(code.ascii_art(syndrome, RotatedPlanarPauli(code, error)))

Error weight: 36.87                                                  
  ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌─Z─┐   ┌───┐      
  ·───X───·───·───·───·───·───·───·───Z───·───·───·─┐
  │░X░│   │░░░│ Z │░░░│   │░░░│   │░░░│ Z │░░░│   │░│
┌─·───·───·───Z───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│ Z │░░░│   │░░░│   │░X░│   │░░░│   │░░░│  
└─·───·───·───·───·───·───·───·───X───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░X░│   │░░░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│   │░░░│ Z │░X░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───·───Y───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░X░│ Z │░X░│   │░░░│ Z │░░░│   │░│
┌─·───·───·───·───·───·───Y───·───·───Z───·───·───·─┘
│░│   │░░░│   │░░░│   │░X░│ Z │░░░│ Z │░░░│   │░X░│  
└─·───·───·───·───·───·───·───·───·───·───·───X───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░░░│   │░X░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│   │░░░│   │░X░│   │░░░│   │░░░│   │░░░│  
└─·───·───·─

# Standard MWPM-decoder recovery proposal

In [16]:
decoder = Matching(
    graph=code.stabilizers
)

recovery = decoder.decode(syndrome)
recovery_syndromes = code.stabilizers @ recovery
recovery_logicals = code.logicals @ recovery

recovery_x, recovery_z = recovery.reshape((2,-1))
recovery_weight = sum([
    wx[jnp.where(jnp.logical_and(recovery_x==1, recovery_z==0))].sum(),
    wy[jnp.where(jnp.logical_and(recovery_x==1, recovery_z==1))].sum(),
    wz[jnp.where(jnp.logical_and(recovery_x==0, recovery_z==1))].sum(),
])
print(f"Recovery weight: {recovery_weight:.2f}")
print(code.ascii_art(recovery_syndromes, RotatedPlanarPauli(code, recovery)))

Recovery weight: 170.43
  ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌─Z─┐   ┌───┐      
  X───·───·───·───·───·───·───·───·───Z───·───·───·─┐
  │░X░│   │░░░│ Z │░░░│   │░░░│   │░░░│ Z │░░░│   │░│
┌─·───·───·───Z───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│ Z │░░░│   │░░░│   │░X░│   │░░░│   │░░░│  
└─·───·───·───·───·───·───·───·───X───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░X░│   │░░░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│   │░░░│ Z │░X░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───·───Y───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░X░│ Z │░X░│   │░░░│ Z │░░░│   │░│
┌─·───·───·───·───·───·───X───·───·───Z───·───·───·─┘
│░│   │░░░│   │░░░│   │░X░│ Z │░░░│ Z │░░░│   │░X░│  
└─·───·───·───·───·───·───Z───·───·───·───·───X───·─┐
  │░░░│   │░░░│   │░░░│ Z │░░░│   │░░░│   │░X░│   │░│
┌─·───·───·───·───·───Z───·───·───·───·───·───·───·─┘
│░│   │░░░│   │░░░│ Z │░X░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───Z───·───X───·───·───·───·───·───·─┐
  │░

# Recursive MWPM-decoder recovery proposal

Method based on the following paper: https://journals.aps.org/pra/pdf/10.1103/PhysRevA.108.022401

In [17]:
recovery = recursive_mwpm(code, syndrome, noise_model, error_probability, noise_permutations, iteration_limit=10, verbose=True)
recovery_x, recovery_z = recovery.reshape((2,-1))
recovery_weight = sum([
    wx[jnp.where(jnp.logical_and(recovery_x==1, recovery_z==0))].sum(),
    wy[jnp.where(jnp.logical_and(recovery_x==1, recovery_z==1))].sum(),
    wz[jnp.where(jnp.logical_and(recovery_x==0, recovery_z==1))].sum(),
])
print(f"Recovery weight: {recovery_weight:.2f}"+50*' ')
print(code.ascii_art(code.stabilizers @ recovery, RotatedPlanarPauli(code, recovery)))

Recovery weight: 161.24                                                  
  ┌───┐   ┌───┐   ┌───┐   ┌───┐   ┌─Z─┐   ┌───┐      
  X───·───·───·───·───·───·───·───·───Z───·───·───·─┐
  │░X░│   │░░░│ Z │░░░│   │░░░│   │░░░│ Z │░░░│   │░│
┌─·───·───·───Z───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│ Z │░░░│   │░░░│   │░X░│   │░░░│   │░░░│  
└─·───·───·───·───·───·───·───·───X───·───·───·───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│   │░X░│   │░░░│   │░│
┌─·───·───·───·───·───·───·───·───·───·───·───·───·─┘
│░│   │░░░│   │░░░│ Z │░X░│   │░░░│   │░░░│   │░░░│  
└─·───·───·───·───·───Y───·───·───·───·───·───·───·─┐
  │░░░│   │░░░│   │░X░│ Z │░X░│   │░░░│ Z │░░░│   │░│
┌─·───·───·───·───·───·───X───·───·───Z───·───·───·─┘
│░│   │░░░│   │░░░│   │░X░│ Z │░░░│ Z │░░░│   │░X░│  
└─·───·───·───·───·───·───·───Z───·───·───·───X───·─┐
  │░░░│   │░░░│   │░░░│   │░░░│ Z │░░░│   │░X░│   │░│
┌─·───·───·───·───·───·───·───Z───·───·───·───·───·─┘
│░│   │░░░│   │░░░│   │░X░│ Z │░░░│   │░░░│   │░X░│  
└─·───·─